# Yellow Taxi Data query in Vast DB - Part 4

In this we verify the data imported into Vast DB can be queried.

<div class="alert alert-block alert-info">
💡 <b>NOTE</b>
<br/>
<ul>
<li>The <b>trino</b> project need to be running for this notebook.</li>
<li>You can run projects with `cd trino && docker compose up -d --wait`.
</div>


In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%load_ext sql
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/vast  --connection_arguments '{"http_scheme":"https", "verify": false}'

Connecting to 'trino://admin@se-var-vastdb-ingest:8443/vast'

In [2]:
%%sql

show schemas in vast

Running query in 'trino://admin@se-var-vastdb-ingest:8443/vast'

Schema
cosmos-db|rag-schema
csnowdb|nyc_data
csnowdb|nyt
csnowdb|social_media
demodbs|auditcopy
information_schema
petika-db|catalog
petika-db|raw
system_schema
vast-audit-log-bucket|vast_audit_log_schema


In [3]:
import os
VASTDB_NYT_BUCKET = os.getenv('VASTDB_NYT_BUCKET')
VASTDB_NYT_SCHEMA = os.getenv('VASTDB_NYT_SCHEMA')
VASTDB_NYT_TABLE = os.getenv('VASTDB_NYT_TABLE')

print(VASTDB_NYT_BUCKET, VASTDB_NYT_SCHEMA, VASTDB_NYT_TABLE)

csnowdb nyt taxi


In [4]:
%%sql
    
show tables in vast."{{VASTDB_NYT_BUCKET}}|{{VASTDB_NYT_SCHEMA}}"

Running query in 'trino://admin@se-var-vastdb-ingest:8443/vast'

Table
taxi


In [5]:
%%sql

select 
    format('%,d', COUNT(*)) as row_count 
from
    vast."{{VASTDB_NYT_BUCKET}}|{{VASTDB_NYT_SCHEMA}}".{{VASTDB_NYT_TABLE}}

Running query in 'trino://admin@se-var-vastdb-ingest:8443/vast'

row_count
"1,763,456,499"


In [6]:
%%sql

SELECT
    *
FROM
    vast."{{VASTDB_NYT_BUCKET}}|{{VASTDB_NYT_SCHEMA}}".{{VASTDB_NYT_TABLE}}
LIMIT 10

Running query in 'trino://admin@se-var-vastdb-ingest:8443/vast'

airport_fee,congestion_surcharge,dolocationid,dropoff_latitude,dropoff_longitude,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,pickup_latitude,pickup_longitude,ratecodeid,store_and_fwd_flag,surcharge,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid
None,None,None,40.804997,-73.96602,None,17.3,None,0.5,2,Cre,None,40.746595,-73.982427,1,None,0.0,3.46,0.0,21.26,2010-06-08 12:26:00,2010-06-08 12:00:00,5.19,VTS
None,None,None,40.75108,-73.89033499999998,None,6.5,None,0.5,5,CAS,None,40.741745,-73.88093,1,None,0.5,0.0,0.0,7.5,2010-06-07 03:58:00,2010-06-07 03:51:00,1.3,VTS
None,None,None,40.749913,-73.99141,None,9.7,None,0.5,2,Cre,None,40.753177,-73.9721,1,None,1.0,2.0,0.0,13.2,2010-06-30 19:25:00,2010-06-30 19:08:00,1.66,VTS
None,None,None,40.76074,-73.998022,None,5.7,None,0.5,1,CAS,None,40.750298,-73.994845,1,None,0.0,0.0,0.0,6.2,2010-06-12 11:23:00,2010-06-12 11:16:00,1.03,VTS
None,None,None,40.77166,-73.947102,None,6.5,None,0.5,2,CAS,None,40.781223,-73.960577,1,None,0.0,0.0,0.0,7.0,2010-06-28 08:40:00,2010-06-28 08:32:00,1.27,VTS
None,None,None,40.725388,-74.00206799999998,None,7.3,None,0.5,3,CAS,None,40.739943,-73.990908,1,None,1.0,0.0,0.0,8.8,2010-06-30 19:25:00,2010-06-30 19:14:00,1.45,VTS
None,None,None,40.777671,-73.94597899999998,None,14.9,None,0.5,1,Cas,None,40.731727,-73.99682,1,None,0.0,0.0,0.0,15.4,2010-06-06 19:58:19,2010-06-06 19:39:14,4.9,CMT
None,None,None,40.792823,-73.974767,None,8.9,None,0.5,5,Cre,None,40.779318,-73.94467299999998,1,None,0.5,2.0,0.0,11.9,2010-06-02 20:23:00,2010-06-02 20:12:00,2.37,VTS
None,None,None,40.702112,-73.982758,None,5.7,None,0.5,2,Cre,None,40.689687,-73.973472,1,None,0.5,2.0,0.0,8.699999999999998,2010-06-09 21:54:00,2010-06-09 21:48:00,1.33,VTS
None,None,None,40.723255,-73.989483,None,9.3,None,0.5,1,Cre,None,40.694827,-73.983642,1,None,0.5,1.7,0.0,12.0,2010-06-06 01:30:00,2010-06-06 01:18:00,2.31,VTS
